# Исследование надежности заемщиков.

**Цель исследования** - проверка четырёх гипотез:

1. Существует зависимость между количеством детей в семье и возвратом кредита в срок.
2. Существует зависимость между семейным положением и возвратом кредита в срок.
3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
4. Существует зависимость между целью кредита и его возвратом в срок.

**Ход исследования**

1. Обзор данных, полученных из файла `datasets.csv`.
2. Предобработка данных.
3. Проверка гипотез.

## Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(10)) # получение первых 10 строк таблицы data
print()
print()
data.info() # получение общей информации о данных в таблице data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 11 столбцов.
Согласно документации к данным:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;  
* `debt` —  имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

**Выводы**

В каждой строке таблицы — данные об одном клиенте. Часть колонок описывает основные характеристики  клиента: пол, возраст, уровень дохода, уровень образования, трудовой стаж, тип занятости, семейное положение, количество детей в семье. Еще две колонки имеют отношение непосредственно к кредиту: цель кредита и факт его возврата в срок.

Можно предположить, что этих данных достаточно для проверки гипотезы, но сначала нужно устранить следующие проблемы:

1. Столбцы `days_employed` и `total_income` содержат пропуски. Их нужно заменить подходящим значением.
2. В столбце `days_employed` содержатся отрицательные значения, что противоречит характеру содержащихся в нем данных (трудовой стаж). Необходимо исправить это и проверить столбец на наличие других аномалий.
3. В столбцах `days_employed`и `total_income`неудачно выбран тип данных: float. Имеет смысл заменить его на int.
4. В столблце `education` некоторые ячейки заполнены капсолоком, что является причиной появления дубликатов. Необходимо исправить это, а также проверить остальные данные на наличие явных и скрытых дубликтов.
5. Данные из столбцов `total_income` и `purpose` для проверки гипотез удобнее объединить в несколько категорий.

## Предобработка данных

### Заполнение пропусков.

Найдем пропущенные значения в столбце `total_income`.

In [2]:
# фильтр для строк в столбце 'total_income',
#содержащих Nan, и их подсчёт
total_income_nan = len(data[data['total_income'].isna()])
display(total_income_nan)
total_income_nan / len(data) * 100

2174

10.099883855981417

In [3]:
# фильтр для строк в столбце 'days_employed',
#содержащих Nan, и их подсчёт
days_employed_nan = len(data[data['days_employed'].isna()])
display(days_employed_nan)
days_employed_nan / len(data) * 100

2174

10.099883855981417

В столбцах с уровнем доходов и трудовым стажем найдено по 2174 значений Nan, что составляет чуть больше 10% от общего количества данных.

Заполним пропущенные значения в столбце `total_income` медианным значением. Этот метод лучше подходит для количественных значений, которые могут содержать "выбросы" (что характерно для уровня доходов).

In [4]:
total_income_median = data['total_income'].median() # вычисление медианного значения
total_income_median

145017.93753253992

In [5]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) # замена Nan на медианное значение

In [6]:
len(data[data['total_income'].isna()]) # проверка

0

### Проверка данных на аномалии и исправления.

In [7]:
# количество отрицательных значений в столбце 'days_employed'
len(data.loc[data['days_employed'] < 0, 'days_employed'])

15906

В столбце 'days_employed' большая часть данных - отрицательные числа. Судя по их количеству, это технологическая ошибка.

In [8]:
# замена отрицательных значений в столбце 'days_employed' на абсолютные
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'].abs()

In [9]:
data['days_employed'].head(15)

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
Name: days_employed, dtype: float64

In [10]:
# проверка, остались ли отрицательные значения в столбце 'days_employed'
len(data.loc[data['days_employed'] < 0, 'days_employed'])

0

In [11]:
# проверка столбца с возрастом клиента на нулевые значения
dob_years_0 = len(data.loc[data['dob_years'] == 0])
display(dob_years_0)
dob_years_0 / len(data) * 100

101

0.4692218350754936

В столбце с возрастом клиентов содержится 101 строка, где возраст равен нулю. Это составляет около 0,4% от общего количества данных, следовательно, этими данными можно пренебречь.

Проверим, насколько трудовой стаж соответствует возрасту клиентов.

In [12]:
# количество клиентов с трудовым стажем более 50 лет
old_employees = len(data.loc[data['days_employed'] > 50*365, 'days_employed'])
display(old_employees)
old_employees / len(data) * 100

3446

16.00929152148664

In [13]:
# количество клиентов старше 70 лет
old_clients = len(data.loc[data['dob_years'] > 70, 'dob_years'])
display(old_clients)
old_clients / len(data) * 100

106

0.4924506387921022

В столбце 'days_employed' выявлено 16% клиентов, чей стаж превышает 50 лет, при этом клиентов, старше 70 лет, всего 0,5%. Значит, в данных есть ошибки.

Проверим столбец с трудовым стажем на наличие других аномалий. Так, в строке с индексом 4 этого столбца указан стаж 340266 дней, т.е. более 900 лет.

In [14]:
# выявление строк с аномальным трудовым стажем (более 100 лет)
len(data.loc[data['days_employed'] > 100*365, 'days_employed'])

3445

Избавимся от аномальных значений. Для этого выявим минимальное значение среди аномальных. Затем напишем функцию, которая заменит все превышающие его значения на максимально возможный трудовой стаж.

In [15]:
# нахождение минимального значения среди всех аномальных (более 100 лет)
data.loc[data['days_employed'] > 100*365, 'days_employed'].min()

328728.72060451825

In [16]:
# функция для замены аномальных значений в столбце 'days_employed' на максимально
# возможный трудовой стаж
max_days_employed = data.loc[data['days_employed'] > 100*365, 'days_employed'].min()

def days_employed_anomaly(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    
    if days_employed > max_days_employed:
        days_employed = (dob_years - 18) * 365
    return days_employed

data['days_employed'] = data.apply(days_employed_anomaly, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь можно вычислить медианное значение для столбца 'days_employed' и заполнить пропуски.

In [17]:
# вычисление медианного значения
days_employed_median = data['days_employed'].median()
days_employed_median

2188.7564450779378

In [18]:
# замена пропусков на медианное значение
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

Проверим остальные столбцы на аномалии.

In [19]:
# проверка столбца "children" на аномалии
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` датасета содержатся аномальные значения: -1 ребенок и 20 детей. Так как их доля невелика, удалим эти строки.

In [20]:
# удаление строк, где детей 20 и -1
data = data.loc[data['children'] != 20]
data = data.loc[data['children'] != -1]
data.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21397,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21398,0,17885.000000,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21399,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21400,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [21]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [22]:
# проверка столбца "dob_years" на аномалии
data['dob_years'].value_counts(ascending=False)

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
24    263
64    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В датасете содержится 100 клиентов, которым 0 лет. Удалим эти строки.

In [23]:
data = data.loc[data['dob_years'] != 0]

In [24]:
data.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21297,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21298,0,17885.000000,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21299,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21300,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [25]:
data['dob_years'].value_counts(ascending=False)

35    614
41    603
40    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
22    183
66    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [26]:
# обзор столбца "total_income"
data['total_income'].describe()

count    2.130200e+04
mean     1.652232e+05
std      9.810088e+04
min      2.066726e+04
25%      1.077766e+05
50%      1.450179e+05
75%      1.955420e+05
max      2.265604e+06
Name: total_income, dtype: float64

### Изменение типов данных.

In [27]:
# приведение столбца 'days_employed' к целочисленному типу
data['days_employed'] = data['days_employed'].astype('int32')

In [28]:
# приведение столбца 'total_income' к целочисленному типу
data['total_income'] = data['total_income'].astype('int32')

In [29]:
data.info() # проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21302 non-null  int64 
 1   days_employed     21302 non-null  int32 
 2   dob_years         21302 non-null  int64 
 3   education         21302 non-null  object
 4   education_id      21302 non-null  int64 
 5   family_status     21302 non-null  object
 6   family_status_id  21302 non-null  int64 
 7   gender            21302 non-null  object
 8   income_type       21302 non-null  object
 9   debt              21302 non-null  int64 
 10  total_income      21302 non-null  int32 
 11  purpose           21302 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


### Удаление дубликатов.

In [2]:
# количество явных дубликатов
data.duplicated().count()

21525

In [3]:
data.duplicated().sum()

54

In [4]:
# выявление неявных дубликатов
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [5]:
# приведение значения столбца 'education' к нижнему регистру
data['education'] = data['education'].str.lower()
data['education'].head(15)

0                  высшее
1                 среднее
2                 среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 среднее
8                  высшее
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
Name: education, dtype: object

In [33]:
data['education'].drop_duplicates().reset_index(drop=True)

0                 высшее
1                среднее
2    неоконченное высшее
3              начальное
4         ученая степень
Name: education, dtype: object

In [34]:
data['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим другие столбцы на наличие скрытых дубликатов.

In [35]:
data['family_status'].value_counts()

женат / замужем          12254
гражданский брак          4139
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

In [36]:
data['income_type'].value_counts()

сотрудник          10996
компаньон           5034
пенсионер           3819
госслужащий         1447
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [6]:
# повторная проверка на наличие дубликатов
data.duplicated().count()

21525

In [7]:
data.duplicated().sum()

71

In [38]:
# удаление явных дубликатов
data = data.drop_duplicates().reset_index(drop=True)

In [39]:
data.duplicated().count()

21231

Явные дубликаты были удалены методом drop_duplicates(). В столбце 'education' были выявлены и удалены дубликаты, вероятнее всего ставшие следствием человеческого фактора (разный регистр).

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [40]:
# создание "словаря" для столбца 'education'
educ_dict = data[['education_id', 'education']]
educ_dict = educ_dict.drop_duplicates().reset_index(drop=True)
educ_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [41]:
# создание "словаря" для столбца 'family_status'
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [42]:
# удаление столбцов 'family_status' и 'education'
data = data.drop(['family_status','education'], axis=1)
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Категоризация дохода.

In [43]:
# функция для группировки клиентов по уровню дохода

def income_group(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

#print (income_group(145018))
#print(income_group(5))
#print(income_group(39284792374))

# добавим новый столбец к датасету
data['total_income_category'] = data['total_income'].apply(income_group)
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21226,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21227,0,17885,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21228,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21229,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Категоризация целей кредита.

In [44]:
# выявление скрытых дубликатов в столбце 'purpose'
data['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

In [45]:
# напишем функцию, которая объединит имеющиеся в датасете цели кредита
# в четыре категории: операции с автомобилем, получение образования,
# проведение свадьбы и операции с недвижимостью
def purpose_group(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    if ('жиль' in purpose) or ('недвиж' in purpose):
        return 'операции с недвижимостью'

# проверка
purpose_group('ремонт жилью')

'операции с недвижимостью'

In [46]:
# добавим к датасету столбец с категоризированными значениями целей кредита
data['purpose_category'] = data['purpose'].apply(purpose_group)
data.head(20)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [47]:
data = data.drop_duplicates().reset_index(drop=True)

In [48]:
# повторное удаление дубликатов
data.duplicated().count()

21231

**Выводы**
Предобработка данных обнаружила следующие проблемы:

- пропущенные и аномальные значения,
- явные и скрытые дубликаты,
- неудачный тип данных,
- неудобно представленные данные.

Пропущенные значения и аномалии были скорректированы, типы данных исправлены, дубликаты удалены. Данные в столбцах `education`, `family_status` и `purpose` были объединены в категории, более удобные для работы.

## Гипотезы

### Влияние количества детей в семье на возврат кредита

Рассчитаем корреляцию между количеством детей в семье и возвратом кредита:

In [49]:
# Расчет коэффициента корреляции Пирсона для столбцов 'children' и 'debt'
data.children.corr(data.debt)

0.024958000127971818

Коэффициент равный 0.02 указывает на отсутствие корреляции между количеством детей в семье и возвратом кредита в срок.

Применим другой метод для проверки этого вывода - построим сводную таблицу для столбцов `children` и `debt`.

In [17]:
children_debt_corr = pd.pivot_table(data, index=['children'], values=['debt'], aggfunc=[sum, len])
children_debt_corr['ratio'] = children_debt_corr['sum'] / children_debt_corr['len'] * 100
children_debt_corr

,sum,len,ratio
,debt,debt,
children,,,
-1,1,47,2.127660
0,1063,14149,7.512898
1,444,4818,9.215442
2,194,2055,9.440389
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000
20,8,76,10.526316


На таблице видно, что разброс между семьями без детей и семьями с четырьмя детьми, не вернувших кредит вовремя - максимум 2%. Можно сделать вывод, что прямой зависимости между количеством детей в семье и возвращением кредита нет.

### Влияние семейного положения клиента на возврат кредита

In [51]:
# Расчет коэффициента корреляции Пирсона для столбцов 'family_status_id' и 'debt'
data.family_status_id.corr(data.debt)

0.020530792656592012

Коэффициент равный 0.02 указывает на отсутствие корреляции между семейным положением и возвратом кредита в срок.

Применим другой метод для проверки этого вывода - построим сводную таблицу для столбцов `family_status_id` и `debt`.

In [52]:
# создание сводной таблицы
family_status_debt_corr = pd.pivot_table(data, index=['family_status_id'], values=['debt'], aggfunc=[sum, len])
family_status_debt_corr['ratio'] = family_status_debt_corr['sum'] / family_status_debt_corr['len'] * 100

# добавим в сводную таблицу "словарь" со значением семейного положения
family_status_debt_dict = family_status_debt_corr.merge(family_status_dict, 
                                                        on='family_status_id', 
                                                        how='left')
family_status_debt_dict

/opt/conda/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,"(sum, debt)","(len, debt)","(ratio, )",family_status
0,0,923,12213,7.557521,женат / замужем
1,1,383,4113,9.311938,гражданский брак
2,2,62,946,6.553911,вдовец / вдова
3,3,84,1179,7.124682,в разводе
4,4,272,2780,9.784173,Не женат / не замужем


Разница между клиентами разного семейного положения, не вернувших кредит в срок, не превышает 3,2%. Вторая гипотеза не подтверждается.

### Влияние уровня доходов на возврат кредита

In [53]:
# Расчет коэффициента корреляции Пирсона для столбцов 'total_income' и 'debt'
data.total_income.corr(data.debt)

-0.011855127061185227

Коэффициент равный 0.01% указывает на отсутствие корреляции между семейным положением и возвратом кредита в срок.

Применим другой метод для проверки этого вывода - построим сводную таблицу для столбцов `total_income` и `debt`.

In [54]:
total_income_debt_corr = pd.pivot_table(data, index=['total_income_category'], values=['debt'], aggfunc=[sum, len])
total_income_debt_corr['ratio'] = total_income_debt_corr['sum'] / total_income_debt_corr['len'] * 100
total_income_debt_corr

,sum,len,ratio
,debt,debt,
total_income_category,,,
A,2,25,8.000000
B,353,4987,7.078404
C,1346,15850,8.492114
D,21,347,6.051873
E,2,22,9.090909


Основные категории клиентов, берущих кредиты - те, чьи доходы располагаются в диапазоне от 200 000 до 1 000 000 и от 50 000 до 200 000. Разница в неплательщиках между первыми и вторыми не превышает 1,5%. Следовательно, третья гипотеза не подтверждается.

### Влияние цели кредита на его возврат

Так как столбец `purpose` представлен данными типа str, применить к нему метод рассчета коэффициента Пирсона невозможно. Построим сводную таблицу.

In [55]:
purpose_debt_corr = pd.pivot_table(data, index=['purpose_category'], values=['debt'], aggfunc=[sum, len])
purpose_debt_corr['ratio'] = purpose_debt_corr['sum'] / purpose_debt_corr['len'] * 100
purpose_debt_corr

,sum,len,ratio
,debt,debt,
purpose_category,,,
операции с автомобилем,397,4258,9.323626
операции с недвижимостью,777,10704,7.258969
получение образования,369,3970,9.294710
проведение свадьбы,181,2299,7.872988


Мы видим, что половина всех кредитов берётся на недвижимость, приблизительно по 20% на автомобили и образование, 10% на проведение свадьбы. Доля не возвративших кредит составляет:

- 7.2% при сделках с недвижимостью;
- 9,4% при операциях с автомобилем;
- 9,2% при кредитах на образование;
- 8% при кредитах на свадьбу.

Разброс между категориями наиболее часто и наиболее редко не возращающих кредиты клиентов не превышает 2,2%.

## Итоги

### Предобработка

В ходе предобработки данных были обнаружены следующие проблемы:
- в столбцах `total_income` и `days_employed` содержится около 10% пропусков. В первом случае пропуски были заполнены медианным значением. Во втором случае сначала была исправлена обнаруженная аномалия (у некоторых клиентов трудовой стаж составлял более 900 лет), лишь затем были заполнены пропуски.
- в столбцах `dob_years` и `children` также были обнаружены аномалии; так как количество аномальных строк было невелико относительно всего датасета, они были удалены.
- в столбцах `days_employed` и `total_income` был изменен тип значений на более подходящий (c float на int).
- в данных были обнаружены как явные, так и неявные дубликаты. Для их устранения в столбце `education` все значения были приведены к единому регистру; данные столбцов `total_income` и  `purpose` были объединены в категории для более удобной работы. После этого из данных были удалены все явные дубликаты.

### Проверка гипотез

Первые три гипотезы были проверены двумя способами - при помощи расчета коэффициента корреляции Пирсона и при помощи построения сводной таблицы. Четвертая гипотеза была проверена при помощи сводной таблицы. По результатам проверки гипотез можно сделать вывод, что общая доля клиентов, не вернувших кредит вовремя, не превышает 10% и не опускается ниже 6%, вне зависимости от семейного положения, количества детей в семье, уровня доходов и целей кредита. Разброс между клиентами вернувшими и не вернувшими кредит слишком незначителен, чтобы можно было делать выводы об однозначном влиянии выделенных характеристик на факт возврата кредита.

Было обнаружено, что семьи без детей не возвращают кредит вовремя в 7,5% случаев, а с одним или двумя детьми - в 9,2% и 9,5% соответственно. Чаще всего не возвращают кредит холостые клиенты (9,8%), чуть реже - клиенты, состоящие в гражданском браке (9,3%); меньше всего долгов у клиентов, состоящих в зарегистрированном браке (7,6%).

Клиенты с ежемесячным доходом от 50 до 200 тысяч не возвращают кредиты вовремя в 8,5% случаев, чуть реже (7,7%) это происходит у клиентов, чей ежемесячный доход составляет от 200 тысяч до 1 миллиона рублей.

Структура целей кредитования следующая:
- 45% на операции с недвижимостью;
- 23% на операции с автомобилем;
- 21,5% на получение образования;
- 10,5% на проведение свадьбы.

Доля не возвративших в этих категория составляет соответственно:

- 7.2% при сделках с недвижимостью;
- 9,4% при операциях с автомобилем;
- 9,2% при кредитах на образование;
- 8% при кредитах на свадьбу.


### Рекомендации

При выдаче кредита необходимо учитывать, что от всех оформленных кредитов от 6% до 10% не будет возвращено в срок. Также нужно учитывать, что чаще задерживают возврат кредита семьи с детьми, клиенты холостые, либо состоящие в гражданском браке. Уровень дохода незначительно, но влияет на возврат кредита, чаще задерживают выплаты клиенты, чей доход ниже двухсот дысяч рублей в месяц. Меньше всего задолженностей у кредитов, взятых на операции с недвижимостью, больше всего - у кредитов на автомобиль и образование.